# Imports

In [502]:
import pandas as pd
from pandas import DataFrame, Series 
import json

# Loading json file

In [503]:
list_of_issues_dict_data = [json.loads(line) for line in open('SPM587SP18issues.json')]

In [504]:
issues_df = DataFrame(list_of_issues_dict_data)

# Converting the dataframe object into json records

In [505]:
s = issues_df.to_json(orient='records')

# Loading the json records into variable

In [506]:
cont = json.loads(s)

# Importing Elasticsearch and helpers of elastic search

In [507]:
from elasticsearch import Elasticsearch, helpers

In [508]:
es = Elasticsearch()

# Inserting the data into Elastic Search (Bulk Insertion)

In [509]:
actions =[]
for data in cont:
    y=data['labels']
    dd = dict()
    for x in y:
        splt = x.split(':')
        l=len(splt)
        if l == 2:
            dd.update({splt[0]:splt[1]})
    data['Longitude'] = str(dd.get('Longitude'))
    data['Latitude'] = str(dd.get('Latitude'))
    action = {
        "_index": "issues",
        "_type": "issue",
        "_id" : data['issue_number'],
        "_source": data
    }
    
    actions.append(action)
    helpers.bulk(es,actions)

In [510]:
pooja = cont[0]
print(pooja['labels'])
dd = {k:v for k,v in (x.split(':') for x in pooja['labels']) }

['Category:Inquiry', 'DetectionPhase:Field', 'OriginationPhase:Coding', 'Priority:High', 'Status:inProgress']


In [511]:
dd['Category']

'Inquiry'

# Qurey to fetch all the data from elastic Search

In [512]:
doc = {
    'size' : 270,
    'query':{
        'match_all':{}
    }
 }
result_without_condition=es.search(index='issues',body=doc, scroll = "1h")


In [513]:
len(doc)
result_without_condition

{'_scroll_id': 'DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAADiFlZwYWNVc0FfUnhDZy0tVnNIbXVFZ2cAAAAAAAAA4xZWcGFjVXNBX1J4Q2ctLVZzSG11RWdnAAAAAAAAAOQWVnBhY1VzQV9SeENnLS1Wc0htdUVnZwAAAAAAAADlFlZwYWNVc0FfUnhDZy0tVnNIbXVFZ2cAAAAAAAAA5hZWcGFjVXNBX1J4Q2ctLVZzSG11RWdn',
 '_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '24',
    '_index': 'issues',
    '_score': 1.0,
    '_source': {'Author': 'HSP18SCM69D',
     'Latitude': '41.891551',
     'Longitude': '-87.607375',
     'State': 'open',
     'closed_at': None,
     'created_at': '2018-04-09',
     'issue_number': 24,
     'labels': ['Address:600 E GRAND AVE',
      'Category:Inquiry',
      'DetectionPhase:Coding',
      'Latitude:41.891551',
      'Longitude:-87.607375',
      'OriginationPhase:Documentation',
      'Priority:Major',
      'Status:pendingReview']},
    '_type': 'issue'},
   {'_id': '22',
    '_index': 'issues',
    '_score': 1.0,
    '_source': {'Author': 'HSP18SCM69D',
     'Latitude': '40.1

In [514]:
sid = result_without_condition['_scroll_id']
scroll_size = result_without_condition['hits']['total']
scroll_size

255

In [515]:
print (sid)
print (scroll_size)

DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAADiFlZwYWNVc0FfUnhDZy0tVnNIbXVFZ2cAAAAAAAAA4xZWcGFjVXNBX1J4Q2ctLVZzSG11RWdnAAAAAAAAAOQWVnBhY1VzQV9SeENnLS1Wc0htdUVnZwAAAAAAAADlFlZwYWNVc0FfUnhDZy0tVnNIbXVFZ2cAAAAAAAAA5hZWcGFjVXNBX1J4Q2ctLVZzSG11RWdn
255


# Populating the data into local variable

In [516]:
final_location_ll = []
while(scroll_size > 0):

    for doc in result_without_condition['hits']['hits']:                   #Iterating each  results of  the qurey 
        location_ll = []
        results = doc['_source']['labels'] 
        for x in results:
            rr1 = x.split(':')
            if rr1[0]== "Latitude":
                lat_val = float(rr1[1])
                location_ll.append(lat_val )
            if rr1[0]== "Longitude":
                lon_val = float(rr1[1])
                location_ll.append(lon_val )
        if (len(location_ll)==2):
            final_location_ll.append(location_ll)
                        
                            #Appending Latitude and Longitude into the list

    result_without_condition = es.scroll(scroll_id = sid, scroll = '2m')    
    sid = result_without_condition['_scroll_id']                                 #Changing the scroll-id
    scroll_size = len(result_without_condition['hits']['hits'])  
final_location_ll

[[41.891551, -87.607375],
 [40.170101, -92.177847],
 [41.883772, -87.625962],
 [41.917164, -87.686965],
 [41.878693, -87.638924],
 [41.838897, -87.646804],
 [41.858415, -87.660926],
 [41.89302, -87.631556],
 [41.878693, -87.638924],
 [41.847095, -87.616767],
 [41.847095, -87.616767],
 [41.847095, -87.616767],
 [41.877817, -87.631247],
 [41.89323, -87.617419],
 [41.847095, -87.616767],
 [41.877846, -87.631296],
 [41.838897, -87.646804],
 [41.838897, -87.646804],
 [41.847095, -87.616767],
 [41.877846, -87.631296],
 [41.878693, -87.638924],
 [41.89323, -87.617419],
 [41.883772, -87.625962],
 [41.917164, -87.686965],
 [41.883772, -87.625962],
 [41.917164, -87.686965],
 [41.880982, -87.630553],
 [41.917164, -87.686965],
 [41.89323, -87.617419],
 [40.170101, -92.177847],
 [41.883772, -87.625962],
 [41.883772, -87.625962],
 [41.880982, -87.630553],
 [41.847095, -87.616767],
 [41.838897, -87.646804],
 [41.89302, -87.631556],
 [41.847095, -87.616767],
 [41.809739, -87.607105],
 [41.847441, -87.

In [517]:
print ("Filtered records for all match :  ", len(final_location_ll))

Filtered records for all match :   137


In [518]:
import folium
from folium import plugins
print(folium.__version__)

0.5.0


In [519]:
without_condition_heatmap = folium.Map([41.90293279, -87.70769386], 
               zoom_start=11)
print(without_condition_heatmap)
without_condition_heatmap

In [520]:
without_condition_heatmap.add_child(plugins.HeatMap(final_location_ll[:40000], radius=15))
without_condition_heatmap

# Query 1 for DetectionPhase is Field AND Priority is Critical

In [521]:
doc1 = {
    'size' : 10000,
    'query':{
        'bool':{
            'must' :[{'match':{'labels':'DetectionPhase:Field'}},{'match':{'labels':'Priority:Critical'}}       
             
            ]
        }
    }
 }
result_with_condition=es.search(index='issues',body=doc1, scroll = "1h")

len(result_with_condition)

#print (result_with_condition)

5

In [522]:
result_with_condition

{'_scroll_id': 'DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAADnFlZwYWNVc0FfUnhDZy0tVnNIbXVFZ2cAAAAAAAAA6BZWcGFjVXNBX1J4Q2ctLVZzSG11RWdnAAAAAAAAAOkWVnBhY1VzQV9SeENnLS1Wc0htdUVnZwAAAAAAAADqFlZwYWNVc0FfUnhDZy0tVnNIbXVFZ2cAAAAAAAAA6xZWcGFjVXNBX1J4Q2ctLVZzSG11RWdn',
 '_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '12',
    '_index': 'issues',
    '_score': 2.680638,
    '_source': {'Author': 'HSP18SCM69D',
     'Latitude': '41.893230',
     'Longitude': 'None',
     'State': 'open',
     'closed_at': None,
     'created_at': '2018-04-09',
     'issue_number': 12,
     'labels': ['Address:645 N MCCLURG CT',
      'Category:Inquiry',
      'DetectionPhase:Field',
      'Latitude:41.893230',
      'OriginationPhase:Coding',
      'Priority:Critical',
      'Status:Approved']},
    '_type': 'issue'},
   {'_id': '52',
    '_index': 'issues',
    '_score': 1.9169197,
    '_source': {'Author': 'HSP18SCM69D',
     'Latitude': '41.893230',
     'Longitude': '-87.617

In [523]:
sid = result_with_condition['_scroll_id']
scroll_size = result_with_condition['hits']['total']
scroll_size

4

# Populating the data into local variable

In [524]:
final_location_ll = []
while(scroll_size > 0):

    for doc in result_with_condition['hits']['hits']:                   #Iterating each  results of  the qurey 
        location_ll = []
        results = doc['_source']['labels'] 
        for x in results:
            rr1 = x.split(':')
            if rr1[0]== "Latitude":
                lat_val = float(rr1[1])
                location_ll.append(lat_val )
            if rr1[0]== "Longitude":
                lon_val = float(rr1[1])
                location_ll.append(lon_val )
        if (len(location_ll)==2):
            final_location_ll.append(location_ll)
                        
                            #Appending Latitude and Longitude into the list

    result_with_condition = es.scroll(scroll_id = sid, scroll = '2m')    
    sid = result_with_condition['_scroll_id']                                 #Changing the scroll-id
    scroll_size = len(result_with_condition['hits']['hits'])  
final_location_ll

[[41.89323, -87.617419], [41.917164, -87.686965], [41.891551, -87.607375]]

In [525]:
print ("Filtered records for query 1 :  ", len(final_location_ll))

Filtered records for query 1 :   3


# For generating HeatMap records fetched based on Query 1

In [526]:
with_condition_heatmap = folium.Map([41.90293279, -87.70769386], 
               zoom_start=11)

with_condition_heatmap.add_child(plugins.HeatMap(final_location_ll[:40000], radius=15))
with_condition_heatmap

# Query 2 for DetectionPhase is Field AND Status is Completed

In [527]:
doc2 = {
    'size' : 10000,
    'query':{
        'bool':{
            'must' :[{'match':{'labels':'DetectionPhase:Field'}},{'match':{'labels':'Status:Completed'}}       
             
            ]
        }
    }
 }
result_with_condition_query2=es.search(index='issues',body=doc2, scroll = "1h")


In [528]:
result_with_condition_query2    
   

{'_scroll_id': 'DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAADsFlZwYWNVc0FfUnhDZy0tVnNIbXVFZ2cAAAAAAAAA7RZWcGFjVXNBX1J4Q2ctLVZzSG11RWdnAAAAAAAAAO4WVnBhY1VzQV9SeENnLS1Wc0htdUVnZwAAAAAAAADwFlZwYWNVc0FfUnhDZy0tVnNIbXVFZ2cAAAAAAAAA7xZWcGFjVXNBX1J4Q2ctLVZzSG11RWdn',
 '_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '22',
    '_index': 'issues',
    '_score': 4.4232726,
    '_source': {'Author': 'HSP18SCM69D',
     'Latitude': '40.170101',
     'Longitude': '-92.177847',
     'State': 'open',
     'closed_at': None,
     'created_at': '2018-04-09',
     'issue_number': 22,
     'labels': ['Address:233 W JACKSON',
      'Category:Bug',
      'DetectionPhase:Field',
      'Latitude:40.170101',
      'Longitude:-92.177847',
      'OriginationPhase:Coding',
      'Priority:High',
      'Status:Completed']},
    '_type': 'issue'},
   {'_id': '42',
    '_index': 'issues',
    '_score': 4.1702557,
    '_source': {'Author': 'HSP18SCM69D',
     'Latitude': '41.891551',

In [529]:
sid = result_with_condition_query2['_scroll_id']
scroll_size = result_with_condition_query2['hits']['total']
scroll_size

4

# Populating the data into local variable

In [530]:
final_location_ll = []
while(scroll_size > 0):

    for doc in result_with_condition_query2['hits']['hits']:                   #Iterating each  results of  the qurey 
        location_ll = []
        results = doc['_source']['labels'] 
        for x in results:
            rr1 = x.split(':')
            if rr1[0]== "Latitude":
                lat_val = float(rr1[1])
                location_ll.append(lat_val )
            if rr1[0]== "Longitude":
                lon_val = float(rr1[1])
                location_ll.append(lon_val )
        if (len(location_ll)==2):
            final_location_ll.append(location_ll)
                        
                            #Appending Latitude and Longitude into the list

    result_with_condition_query2 = es.scroll(scroll_id = sid, scroll = '2m')    
    sid = result_with_condition_query2['_scroll_id']                                 #Changing the scroll-id
    scroll_size = len(result_with_condition_query2['hits']['hits'])  
final_location_ll

[[40.170101, -92.177847],
 [41.891551, -87.607375],
 [41.89323, -87.617419],
 [41.853136, -87.63316]]

In [531]:
print ("Filtered records for query 2 :  ", len(final_location_ll))

Filtered records for query 2 :   4


# For generating HeatMap records fetched based on Query 2

In [532]:
with_condition_heatmap_query2 = folium.Map([41.90293279, -87.70769386], 
               zoom_start=11)

with_condition_heatmap_query2.add_child(plugins.HeatMap(final_location_ll[:40000], radius=15))
with_condition_heatmap_query2

# Query 3 for DetectionPhase is Field AND Priority is Critical AND Status is Approved

In [533]:
doc3 = {
    'size' : 10000,
    'query':{
        'bool':{
            'must' :[{'match':{'labels':'DetectionPhase:Field'}},{'match':{'labels':'Priority:Critical'}},
                     {'match':{'labels':'Status:Approved'}}
             
            ]
        }
    }
 }
result_with_condition_query3=es.search(index='issues',body=doc3, scroll = "1h")


In [534]:
result_with_condition_query3

{'_scroll_id': 'DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAADxFlZwYWNVc0FfUnhDZy0tVnNIbXVFZ2cAAAAAAAAA8hZWcGFjVXNBX1J4Q2ctLVZzSG11RWdnAAAAAAAAAPMWVnBhY1VzQV9SeENnLS1Wc0htdUVnZwAAAAAAAAD1FlZwYWNVc0FfUnhDZy0tVnNIbXVFZ2cAAAAAAAAA9BZWcGFjVXNBX1J4Q2ctLVZzSG11RWdn',
 '_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '12',
    '_index': 'issues',
    '_score': 3.1718001,
    '_source': {'Author': 'HSP18SCM69D',
     'Latitude': '41.893230',
     'Longitude': 'None',
     'State': 'open',
     'closed_at': None,
     'created_at': '2018-04-09',
     'issue_number': 12,
     'labels': ['Address:645 N MCCLURG CT',
      'Category:Inquiry',
      'DetectionPhase:Field',
      'Latitude:41.893230',
      'OriginationPhase:Coding',
      'Priority:Critical',
      'Status:Approved']},
    '_type': 'issue'},
   {'_id': '26',
    '_index': 'issues',
    '_score': 2.7237563,
    '_source': {'Author': 'HSP18SCM69D',
     'Latitude': '41.917164',
     'Longitude': '-87.68

In [535]:
sid = result_with_condition_query3['_scroll_id']
scroll_size = result_with_condition_query3['hits']['total']
scroll_size

3

# Populating the data into local variable

In [536]:
final_location_ll = []
while(scroll_size > 0):

    for doc in result_with_condition_query3['hits']['hits']:                   #Iterating each  results of  the qurey 
        location_ll = []
        results = doc['_source']['labels'] 
        for x in results:
            rr1 = x.split(':')
            if rr1[0]== "Latitude":
                lat_val = float(rr1[1])
                location_ll.append(lat_val )
            if rr1[0]== "Longitude":
                lon_val = float(rr1[1])
                location_ll.append(lon_val )
        if (len(location_ll)==2):
            final_location_ll.append(location_ll)
                        
                            #Appending Latitude and Longitude into the list

    result_with_condition_query3 = es.scroll(scroll_id = sid, scroll = '2m')    
    sid = result_with_condition_query3['_scroll_id']                                 #Changing the scroll-id
    scroll_size = len(result_with_condition_query3['hits']['hits'])  
final_location_ll

[[41.917164, -87.686965], [41.891551, -87.607375]]

In [537]:
print ("Filtered records for query 3 : ", len(final_location_ll))

Filtered records for query 3 :  2


# For generating HeatMap records fetched based on Query 3

In [538]:
with_condition_heatmap_query3 = folium.Map([41.90293279, -87.70769386], 
               zoom_start=11)

with_condition_heatmap_query3.add_child(plugins.HeatMap(final_location_ll[:40000], radius=15))
with_condition_heatmap_query3


# Query 4 for DetectionPhase is Field AND Priority is Critical or High AND Status is Approved or inProgress

In [539]:
doc4 = {
    'size' : 10000,
    'query':{'bool':{'must' :[{'match':{'labels':'DetectionPhase:Field'}},{ 'bool': { 
        'should': [{'match': {'labels': 'Priority:Critical'}},{'match':{'labels':'Priority:High'}}]} },
                { 'bool': { 'should': [{'match': {'labels': 'Status:Approved'}},{'match':{'labels':'Status:inProgress'}}]} }
            ]}} }
result_with_condition_query4=es.search(index='issues',body=doc4, scroll = "1h")
#print (result_with_condition)

In [540]:
sid = result_with_condition_query4['_scroll_id']
scroll_size = result_with_condition_query4['hits']['total']
scroll_size

24

# Populating the data into local variable

In [541]:
final_location_ll = []
while(scroll_size > 0):

    for doc in result_with_condition_query4['hits']['hits']:                   #Iterating each  results of  the qurey 
        location_ll = []
        results = doc['_source']['labels'] 
        for x in results:
            rr1 = x.split(':')
            if rr1[0]== "Latitude":
                lat_val = float(rr1[1])
                location_ll.append(lat_val )
            if rr1[0]== "Longitude":
                lon_val = float(rr1[1])
                location_ll.append(lon_val )
        if (len(location_ll)==2):
            final_location_ll.append(location_ll)
                        
                            #Appending Latitude and Longitude into the list

    result_with_condition_query4 = es.scroll(scroll_id = sid, scroll = '2m')    
    sid = result_with_condition_query4['_scroll_id']                                 #Changing the scroll-id
    scroll_size = len(result_with_condition_query4['hits']['hits'])  
final_location_ll



[[41.89323, -87.617419],
 [41.853136, -87.63316],
 [41.917164, -87.686965],
 [41.891551, -87.607375]]

In [542]:
print ("Filtered records for query 4 :  ", len(final_location_ll))

Filtered records for query 4 :   4


# For generating HeatMap records fetched based on Query 4

In [543]:
with_condition_heatmap_query4= folium.Map([41.90293279, -87.70769386], 
               zoom_start=11)
with_condition_heatmap_query4.add_child(plugins.HeatMap(final_location_ll[:40000], radius=15))
with_condition_heatmap_query4

# Query 5 for Create Folium HeatMap for all locations that got at least 5 issues create for the same location (latitude, Longitude)

In [544]:
doc = {
    'size' : 0,
    'query':{
        'bool':{
            'must_not' :[{'match':{'Latitude':'None'}},{'match':{'Longitude':'None'}} ]} },
   "aggs" : {"latitude": {"terms" : { "field" : "Latitude.keyword" ,"min_doc_count": 5,"order" : { "_count" : "asc" }},
            "aggs":{"longitude":{"terms" : { "field" : "Longitude.keyword","min_doc_count": 5}}}            }
           }}
result_with_condition_query5=es.search(index='issues',body=doc)
result_with_condition_query5

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'aggregations': {'latitude': {'buckets': [{'doc_count': 5,
     'key': '40.170101',
     'longitude': {'buckets': [{'doc_count': 5, 'key': '-92.177847'}],
      'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0}},
    {'doc_count': 5,
     'key': '41.877846',
     'longitude': {'buckets': [{'doc_count': 5, 'key': '-87.631296'}],
      'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0}},
    {'doc_count': 5,
     'key': '41.878693',
     'longitude': {'buckets': [{'doc_count': 5, 'key': '-87.638924'}],
      'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0}},
    {'doc_count': 5,
     'key': '41.891551',
     'longitude': {'buckets': [{'doc_count': 5, 'key': '-87.607375'}],
      'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0}},
    {'doc_count': 6,
     'key': '41.853136',
     'longitude': {'buckets': [{'doc_count': 6, 'key': '-87.633160'}],
      'doc

# Populating the data into local variable

In [545]:
list_query_5 = []
for ele in result_with_condition_query5['aggregations']['latitude']['buckets']:
    lst_q5 = []
    d = dict(ele)
    lst_q5.append(float(d['key']))
    d1 = dict(d['longitude'])
    for i in d1['buckets']:
        dict(i)
        lst_q5.append(float(i['key']))
    list_query_5.append(lst_q5)

# For generating HeatMap records fetched based on Query 5

In [546]:
with_condition_heatmap_quer5 = folium.Map([41.9029,-87.7070],zoom_start=11)
print("Filtered records for query 5 :  ",len(list_query_5))

Filtered records for query 5 :   10


In [547]:
with_condition_heatmap_quer5.add_child(plugins.HeatMap(list_query_5[:40000], radius=15))
with_condition_heatmap_quer5